In [1]:
# 1. Install required libraries
!pip install dash plotly scikit-learn openpyxl dash-bootstrap-components -q

import pandas as pd
import numpy as np
import threading
from datetime import datetime
import plotly.graph_objs as go
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from google.colab import output
import warnings

warnings.filterwarnings('ignore')
print("✅ Libraries ready.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 10.7 MB/s eta 0:00:00
✅ Libraries ready.


In [3]:
# 1. Load Data (Ensure your file is in the /content/ folder)
data = pd.read_excel('/content/Dataset_FMCS_LORA.xlsx')

# 2. Optimized Feature Engineering (No slow loops)
data['is_retry'] = data['is_retry'].fillna(0)
data['creation_date'] = pd.to_datetime(data['creation_date'])
data['time_diff'] = data.groupby('dev_id')['creation_date'].diff().dt.total_seconds().fillna(0)
data['transmission_frequency'] = 1 / (data['time_diff'] + 1)
data['hour'] = data['creation_date'].dt.hour

np.random.seed(42)
data['rssi'] = -70 + np.random.normal(0, 10, len(data))
data['auth_failure_rate'] = data.groupby('dev_id')['is_retry'].transform(lambda x: x.rolling(10, 1).mean())
data['counter_anomaly'] = (data.groupby('dev_id')['counter'].diff() > 1).astype(int)
data['port_change'] = (data.groupby('dev_id')['port'].diff() != 0).astype(int)

# Encoding
le_dev = LabelEncoder()
data['dev_id_encoded'] = le_dev.fit_transform(data['dev_id'].astype(str))
data = data.fillna(0)

# Features
features = ['transmission_frequency', 'rssi', 'time_diff', 'auth_failure_rate',
            'counter_anomaly', 'port_change', 'hour', 'counter', 'dev_id_encoded', 'port']
data['next_time_diff'] = data.groupby('dev_id')['time_diff'].shift(-1)
model_data = data.dropna(subset=['next_time_diff'])
X, y = model_data[features], model_data['next_time_diff']

# Fast Training
print("🚀 Training models... please wait a few seconds.")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# n_estimators=50 is fast and accurate enough for this demo
rf_model = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X_scaled, y)

iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(X_scaled)

# Pre-calculate metrics
y_pred = rf_model.predict(X_scaled)
feature_imp = pd.DataFrame({'feature': features, 'importance': rf_model.feature_importances_}).sort_values('importance', ascending=False)
error_val = mean_absolute_error(y, y_pred)

print(f"✅ Models trained. MAE: {error_val:.4f}")

🚀 Training models... please wait a few seconds.
✅ Models trained. MAE: 81.9874


In [4]:
# --- FULL DASHBOARD CELL: REPLACE EVERYTHING IN YOUR PREVIOUS CELL ---
import dash_bootstrap_components as dbc
from dash import Dash, html, dcc, Input, Output, State
import plotly.graph_objects as go
import numpy as np
import threading
from datetime import datetime

# Initialize App
app = Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

app.layout = dbc.Container([
    html.H1("🛡️ IIoT Security Dashboard", className="text-center text-primary mb-4"),

    # This banner will change color and text
    dbc.Alert("System Online", id="alert", color="info", className="mb-3"),

    dbc.Card([
        dbc.CardHeader("📡 Signal Input"),
        dbc.CardBody([
            dbc.Row([
                dbc.Col([html.Label("Device:"), dcc.Dropdown(id='dev', options=[{'label': d, 'value': d} for d in data['dev_id'].unique()], value=data['dev_id'].iloc[0])], width=4),
                dbc.Col([html.Label("RSSI:"), dcc.Input(id='rssi', type='number', value=-70, className="form-control")], width=4),
                dbc.Col([html.Label("Time Diff:"), dcc.Input(id='time', type='number', value=10, className="form-control")], width=4)
            ]),
            dbc.Button("🔍 Analyze", id="btn", color="primary", className="w-100 mt-3")
        ])
    ], className="mb-3"),

    dbc.Row([
        # Status box with dynamic text and color
        dbc.Col(dbc.Card([dbc.CardHeader("Status"), dbc.CardBody(html.Div(id='anom', style={'fontSize': '22px'}))])),
        dbc.Col(dbc.Card([dbc.CardHeader("Prediction"), dbc.CardBody(html.Div(id='pred'))]))
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id='imp_graph'), width=6),
        dbc.Col(dcc.Graph(id='ts_graph'), width=6)
    ], className="mt-3")
], fluid=True, style={'padding': '20px', 'backgroundColor': '#121212'})

@app.callback(
    [Output('pred', 'children'),
     Output('anom', 'children'),
     Output('anom', 'style'), # For explicit text color
     Output('alert', 'children'),
     Output('alert', 'color'), # For banner color
     Output('imp_graph', 'figure'),
     Output('ts_graph', 'figure')],
    Input('btn', 'n_clicks'),
    [State('dev', 'value'), State('rssi', 'value'), State('time', 'value')]
)
def update_view(n, dev, rssi, time_val):
    if n is None:
        return "0.00s", "READY", {"color": "white"}, "System Online", "info", go.Figure(), go.Figure()

    # 1. PHYSICAL THRESHOLDS (The Manual Guardrail)
    # Real-life RSSI for IoT (like Heltec) is roughly -120 to -30.
    # Anything outside this is an anomaly.
    SAFE_RSSI_MIN = -120
    SAFE_RSSI_MAX = -30

    # 2. MODEL PREDICTION
    dev_enc = le_dev.transform([str(dev)])[0]
    inp = np.array([[1/(time_val+1), rssi, time_val, 0, 0, 0, datetime.now().hour, 100, dev_enc, 1]])
    inp_s = scaler.transform(inp)

    p = rf_model.predict(inp_s)[0]
    ai_is_anom = iso_forest.predict(inp_s)[0] == -1

    # 3. FINAL DECISION (Hybrid Logic)
    # It is an anomaly if the AI says so OR if the RSSI is impossible
    is_anom = ai_is_anom or (rssi < SAFE_RSSI_MIN) or (rssi > SAFE_RSSI_MAX)

    # 4. COLOR AND UI LOGIC
    if is_anom:
        status_msg = "⚠️ ANOMALY"
        status_style = {"color": "#FF4136", "fontWeight": "bold"} # Bright Red
        banner_color = "danger"
    else:
        status_msg = "✅ NORMAL"
        status_style = {"color": "#2ECC40", "fontWeight": "bold"} # Bright Green
        banner_color = "success"

    # Graphs
    fig_imp = go.Figure(go.Bar(x=feature_imp['importance'], y=feature_imp['feature'], orientation='h'))
    fig_imp.update_layout(template="plotly_dark", title="Feature Importance", height=300)

    fig_ts = go.Figure(go.Scatter(y=y[:50], mode='lines'))
    fig_ts.update_layout(template="plotly_dark", title="Historical Traffic", height=300)

    return f"{p:.2f}s", status_msg, status_style, f"Status: {status_msg}", banner_color, fig_imp, fig_ts

# --- START SERVER ---
def run_app():
    app.run(host='0.0.0.0', port=8050, debug=False, use_reloader=False)

# Re-run protection
if 'thread' not in globals() or not thread.is_alive():
    thread = threading.Thread(target=run_app)
    thread.daemon = True
    thread.start()

print("Dashboard Updated! Restart your session if changes don't appear.")
output.serve_kernel_port_as_iframe(8050, height='800')


Dash is running on http://0.0.0.0:8050/

Dashboard Updated! Restart your session if changes don't appear.


INFO:dash.dash:Dash is running on http://0.0.0.0:8050/



<IPython.core.display.Javascript object>